In [1]:
import hail as hl
import requests
import datetime
import subprocess
import pandas as pd
import argparse
#from hail.experimental.ldscsim import simulate_phenotypes
url = 'https://raw.githubusercontent.com/nikbaya/ldscsim/master/ldscsim.py'
r = requests.get(url).text
exec(r)
calculate_phenotypes = calculate_phenotypes
simulate_phenotypes = simulate_phenotypes


wd = 'gs://nbaya/risk_gradients/'


In [2]:
hl.init(log='/tmp/foo.log')

Running on Apache Spark version 2.4.4
SparkUI available at http://ukbb-nb-m.c.ukbb-round2.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.25-04344d214361
LOGGING: writing to /tmp/foo.log


In [11]:
def get_corr_pt(variant_set, maf, h2, pi, use_1kg_eur_hm3_snps, chrom, is_pruned,
                threshold=None, max_reps=60):
    # read in testing mt
    mt = hl.read_matrix_table(
        wd+f'genotypes{("" if chrom is "all" else f".chr{chrom}")}.all.{variant_set}.maf_{maf}{".1kg_eur_hm3" if use_1kg_eur_hm3_snps else ""}.mt')
    n_train = int(300e3)
    seed = 1
    train = hl.import_table(
        wd+f'iid.sim.train.n_{n_train}.seed_{seed}.tsv.bgz').key_by('s')
    test = mt.filter_cols(hl.is_defined(train[mt.s]), keep=False)
    sim_cols = hl.read_table(
        wd+f'sim.cols{("" if chrom is "all" else f".chr{chrom}")}.all.{variant_set}.maf_{maf}.h2_{h2}.pi_{pi}{".1kg_eur_hm3" if use_1kg_eur_hm3_snps else ""}.ht')
    test = test.annotate_cols(sim_y=sim_cols[test.s].y)
    ct_cols = test.count_cols()
    print(f'\n###############\ntest mt col count: {ct_cols}\n###############')

    if is_pruned:

        # define the set of SNPs
        pruned_snps_file = 'ukb_imp_v3_pruned.bim' #from Robert Maier (pruning threshold=0.2)
        variants = hl.import_table(
            wd+pruned_snps_file, delimiter='\t', no_header=True, impute=True)
        print(f'\n... Pruning to variants in {wd+pruned_snps_file}...')
        variants = variants.rename(
            {'f0': 'chr', 'f1': 'rsid', 'f3': 'pos'}).key_by('rsid')
        test = test.key_rows_by('rsid')
        # filter to variants defined in variants table
        test = test.filter_rows(hl.is_defined(variants[test.rsid]))
        ct_rows = test.count_rows()
        print(f'\n###############\ntest mt row count after pruning filter: {ct_rows}\n###############')
    else:
        print(f'\n... Not pruning because is_pruned={is_pruned} ...')

#     if threshold!=None:
# #         ss = hl.import_table(wd+f'sim.gwas.h2_obs_0.765.pi_obs_0.001.n_train_sub_100000.subset_0.1kg_eur_hm3.v2.tsv.gz',
# #                                  impute=True,
# #                                  force_bgz=True)
#         ss = hl.read_table('gs://nbaya/risk_gradients/sim.rows.chr22.all.qc_pos.maf_0.05.h2_0.75.pi_0.01.1kg_eur_hm3.ht/')
#         ss = ss.rename({'rsid':'SNP','A1_1kg':'A1','beta':'BETA'})
#         ss = ss.annotate(P = 0)
#         ss = ss.key_by('SNP')
#         test = test.filter_rows(hl.is_defined(ss[test.rsid]))
#         ct_rows = test.count_rows()
#         print(f'\n###############\ntest mt row count after checking against GWAS for 100k: {ct_rows}\n###############')
    
#         threshold = 1
    
#         print(
#             f'\n###############\npval threshold: pval<{threshold}\n###############')
#         test = test.annotate_rows(P=ss[test.rsid].P)
#         test = test.filter_rows(test.P < threshold)
#         ct_rows = test.count_rows()
#         print(
#             f'\n###############\ntest mt row count after keeping pval<{threshold}: {ct_rows}\n###############')

    n_train_subs = [100e3, 50e3, 20e3, 10e3, 5e3]  # 20e3,
    n_train_subs = [int(x) for x in n_train_subs]
    n_train_sub_ls = []
    subset_ls = []
    r_ls = []  # list of correlation coefficients
    
    n_train = 300e3
    sim_beta = hl.read_table('gs://nbaya/risk_gradients/sim.rows.chr22.all.qc_pos.maf_0.05.h2_0.75.pi_0.01.1kg_eur_hm3.ht/')
    sim_beta = sim_beta.rename({'rsid':'SNP','beta':'true_beta'})
    sim_beta = sim_beta.key_by('SNP')
    print(sim_beta.describe())

    for n_train_sub in n_train_subs:
        # number of training subsets in full training set with n_train_sub individuals each
        n_subsets = int(n_train/n_train_sub)
        for subset in range(n_subsets)[:max_reps]:
#            ss_path = wd + \
#                f'sim.gwas.h2_obs_0.765.pi_obs_0.001.n_train_sub_{n_train_sub}.subset_{subset}{".1kg_eur_hm3" if use_1kg_eur_hm3_snps else ""}.v2.tsv.gz'
#            ss_path = wd+f'sim.chr22.gwas.h2_obs_0.485.pi_obs_0.0009.n_train_sub_{n_train_sub}.subset_{subset}.1kg_eur_hm3.v2.tsv.gz'
            ss_path = wd+f'sim.chr22.gwas.h2_obs_0.7.pi_obs_0.0084.n_train_sub_{n_train_sub}.subset_{subset}.1kg_eur_hm3.v2.tsv.gz'
#             ss_path = wd+f'/sim.chr22.gwas.h2_obs_0.746.pi_obs_1.0.n_train_sub_{n_train_sub}.subset_{subset}.1kg_eur_hm3.v2.tsv.gz'
#             print(
#                 f'\n#############\nn_train_sub={n_train_sub} (subset {subset+1} of {n_subsets})\nss path: {ss_path}\n#############')
            try:
                ss = hl.import_table(ss_path,impute=True,force=True,key='SNP')
            except:
                print(f'file does not exist: {ss_path}')
                next
            ss = ss.rename({'BETA':'gwas_beta'})
            ss = ss.annotate(BETA = (sim_beta[ss.SNP].true_beta!=0)*ss.gwas_beta)
            
            test1 = test.annotate_rows(adj_beta=ss[test.rsid].BETA,
                                       prs_A1=ss[test.rsid].A1)
            test1 = test1.annotate_rows(adj_beta_flipped=(test1.adj_beta*(test1.alleles[0] == test1.prs_A1) +
                                                          -test1.adj_beta*(test1.alleles[0] != test1.prs_A1)))  # flip sign if A1 is not the same in both
            test1 = calculate_phenotypes(
                mt=test1, genotype=test1.dosage, beta=test1.adj_beta_flipped, h2=1)
            test1 = test1.rename({'y': 'prs'})
            r = test1.aggregate_cols(hl.agg.corr(test1.sim_y, test1.prs))
            n_train_sub_ls.append(n_train_sub)
            subset_ls.append(subset)
            r_ls.append(r)
            print(
                f'\n#############\nr for subset {subset+1} of {n_subsets} (n_train_sub={n_train_sub}): {r}\nr2 for subset {subset+1} of {n_subsets}: {r**2}\n#############')
    df = pd.DataFrame(data=list(zip(n_train_sub_ls, subset_ls, [ct_cols]*len(n_train_sub_ls), r_ls)),
                          columns=['n_train', 'subset_id', 'n_test', 'r'])
    print(df)

    hl.Table.from_pandas(df).export(wd+f'prs_cs/corr.pt{".is_pruned" if is_pruned else ""}.threshold_{threshold}.{variant_set}.maf_{maf}.h2_{h2}.pi_{pi}{".1kg_eur_hm3" if use_1kg_eur_hm3_snps else ""}_truly_causal_marginal_betas.tsv')


In [7]:
variant_set = 'qc_pos'
maf=0.05
h2=0.75
pi=0.01
use_1kg_eur_hm3_snps=True
is_pruned=False
chrom=22
max_reps=6

In [12]:
get_corr_pt(variant_set=variant_set, maf=maf, h2=h2, pi=pi,
            use_1kg_eur_hm3_snps=use_1kg_eur_hm3_snps,is_pruned=is_pruned,
            threshold=1,chrom=chrom,max_reps=max_reps)

2019-12-05 14:15:14 Hail: INFO: Reading table with no type imputation
  Loading column 's' as type 'str' (type not specified)
  Loading column 'label_100000' as type 'str' (type not specified)
  Loading column 'label_50000' as type 'str' (type not specified)
  Loading column 'label_20000' as type 'str' (type not specified)
  Loading column 'label_10000' as type 'str' (type not specified)
  Loading column 'label_5000' as type 'str' (type not specified)




###############
test mt col count: 61144
###############

... Not pruning because is_pruned=False ...
----------------------------------------
Global fields:
    'ldscsim': struct {
        h2: float64, 
        pi: array<float64>
    } 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'SNP': str 
    'varid': str 
    'A1_1kg': str 
    'A2_1kg': str 
    'AF': float64 
    'true_beta': float64 
----------------------------------------
Key: ['SNP']
----------------------------------------
None


2019-12-05 14:15:26 Hail: INFO: Reading table to impute column types
2019-12-05 14:15:26 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'BETA' as type 'float64' (imputed)
  Loading column 'P' as type 'float64' (imputed)
2019-12-05 14:15:47 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:15:47 Hail: INFO: Coerced sorted dataset
2019-12-05 14:15:51 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:16:00 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:16:03 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:16:03 Hail: INFO: Coerced sorted dataset
2019-12-05 14:16:04 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:16:11 Hail: INFO: Ordering unsorted dataset with network shuffle



#############
r for subset 1 of 3 (n_train_sub=100000): 0.7751967188739658
r2 for subset 1 of 3: 0.6009299529529624
#############


2019-12-05 14:17:23 Hail: INFO: Reading table to impute column types
2019-12-05 14:17:23 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'BETA' as type 'float64' (imputed)
  Loading column 'P' as type 'float64' (imputed)
2019-12-05 14:17:47 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:17:47 Hail: INFO: Coerced sorted dataset
2019-12-05 14:17:48 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:17:55 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:17:56 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:17:56 Hail: INFO: Coerced sorted dataset
2019-12-05 14:17:57 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:18:03 Hail: INFO: Ordering unsorted dataset with network shuffle


...


#############
r for subset 4 of 6 (n_train_sub=50000): 0.7722259571563674
r2 for subset 4 of 6: 0.5963329289060678
#############


2019-12-05 14:27:40 Hail: INFO: Reading table to impute column types
2019-12-05 14:27:40 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'BETA' as type 'float64' (imputed)
  Loading column 'P' as type 'float64' (imputed)
2019-12-05 14:27:58 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:27:58 Hail: INFO: Coerced sorted dataset
2019-12-05 14:27:59 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:28:05 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:28:06 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:28:06 Hail: INFO: Coerced sorted dataset
2019-12-05 14:28:07 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-05 14:28:13 Hail: INFO: Ordering unsorted dataset with network shuffle



#############
r for subset 5 of 6 (n_train_sub=50000): 0.7710868853835959
r2 for subset 5 of 6: 0.5945749848105748
#############
file does not exist: gs://nbaya/risk_gradients/sim.chr22.gwas.h2_obs_0.7.pi_obs_0.0084.n_train_sub_50000.subset_5.1kg_eur_hm3.v2.tsv.gz


2019-12-05 14:29:25 Hail: WARN: 'gs://nbaya/risk_gradients/sim.chr22.gwas.h2_obs_0.7.pi_obs_0.0084.n_train_sub_50000.subset_5.1kg_eur_hm3.v2.tsv.gz' refers to no files


ValueError: Cannot rename 'BETA' to 'gwas_beta': field already exists.

### Line by line

In [4]:
# read in testing mt
mt = hl.read_matrix_table(
    wd+f'genotypes{("" if chrom is "all" else f".chr{chrom}")}.all.{variant_set}.maf_{maf}{".1kg_eur_hm3" if use_1kg_eur_hm3_snps else ""}.mt')
n_train = int(300e3)
seed = 1
train = hl.import_table(
    wd+f'iid.sim.train.n_{n_train}.seed_{seed}.tsv.bgz').key_by('s')
test = mt.filter_cols(hl.is_defined(train[mt.s]), keep=False)
sim_cols = hl.read_table(
    wd+f'sim.cols{("" if chrom is "all" else f".chr{chrom}")}.all.{variant_set}.maf_{maf}.h2_{h2}.pi_{pi}{".1kg_eur_hm3" if use_1kg_eur_hm3_snps else ""}.ht')
test = test.annotate_cols(sim_y=sim_cols[test.s].y)
# ct_cols = test.count_cols()

2019-12-04 13:03:27 Hail: INFO: Reading table with no type imputation
  Loading column 's' as type 'str' (type not specified)
  Loading column 'label_100000' as type 'str' (type not specified)
  Loading column 'label_50000' as type 'str' (type not specified)
  Loading column 'label_20000' as type 'str' (type not specified)
  Loading column 'label_10000' as type 'str' (type not specified)
  Loading column 'label_5000' as type 'str' (type not specified)



In [33]:
n_train_subs = [100e3, 50e3, 20e3, 10e3, 5e3]  # 20e3,
n_train_subs = [int(x) for x in n_train_subs]
n_train_sub_ls = []
subset_ls = []
r_ls = []  # list of correlation coefficients

n_train = 300e3
sim_beta = hl.read_table('gs://nbaya/risk_gradients/sim.rows.chr22.all.qc_pos.maf_0.05.h2_0.75.pi_0.01.1kg_eur_hm3.ht/')
sim_beta = sim_beta.annotate(A1 = sim_beta.alleles[0])
sim_beta = sim_beta.rename({'rsid':'SNP','beta':'true_beta'})
sim_beta = sim_beta.key_by('SNP')
print(sim_beta.describe())

----------------------------------------
Global fields:
    'ldscsim': struct {
        h2: float64, 
        pi: array<float64>
    } 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'SNP': str 
    'varid': str 
    'A1_1kg': str 
    'A2_1kg': str 
    'AF': float64 
    'true_beta': float64 
    'A1': str 
----------------------------------------
Key: ['SNP']
----------------------------------------
None


In [35]:
n_train_sub=5000
# number of training subsets in full training set with n_train_sub individuals each
subset=0
# ss_path = wd+f'sim.chr22.gwas.h2_obs_0.7.pi_obs_0.0084.n_train_sub_{n_train_sub}.subset_{subset}.1kg_eur_hm3.v2.tsv.gz'
# ss = hl.import_table(ss_path,impute=True,force=True,key='SNP')
# ss = ss.rename({'BETA':'gwas_beta'})
ss=sim_beta.rename({'true_beta':'BETA'})
test1 = test.annotate_rows(adj_beta=ss[test.rsid].BETA,
                           prs_A1=ss[test.rsid].A1)
test1 = test1.annotate_rows(adj_beta_flipped=(test1.adj_beta*(test1.alleles[0] == test1.prs_A1) +
                                              -test1.adj_beta*(test1.alleles[0] != test1.prs_A1)))  # flip sign if A1 is not the same in both
test1 = calculate_phenotypes(mt=test1, genotype=test1.dosage, beta=test1.adj_beta_flipped, h2=1)
test1 = test1.rename({'y': 'prs'})
r = test1.aggregate_cols(hl.agg.corr(test1.sim_y, test1.prs))
n_train_sub_ls.append(n_train_sub)
subset_ls.append(subset)
r_ls.append(r)

2019-12-04 14:08:34 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:08:36 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:08:46 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:08:47 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:08:48 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:08:52 Hail: INFO: Ordering unsorted dataset with network shuffle


In [37]:
r

0.8363386698641746

In [38]:
r**2

0.6994623707101768

In [39]:
test1 = test1.annotate_cols(y_no_noise = sim_cols[test1.s].y_no_noise)

In [40]:
r = test1.aggregate_cols(hl.agg.corr(test1.prs, test1.y_no_noise))

2019-12-04 14:12:59 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:13:01 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:13:05 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:13:08 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:13:09 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:13:13 Hail: INFO: Ordering unsorted dataset with network shuffle


In [43]:
sim_cols.aggregate(hl.agg.stats(sim_cols.y_no_noise)).stdev**2/sim_cols.aggregate(hl.agg.stats(sim_cols.y)).stdev**2

0.6995533238122902

In [41]:
r**2

0.9999895054076228

In [36]:
test1.filter_rows(test1.adj_beta!=0).select_rows('prs_A1','adj_beta','adj_beta_flipped').rows().show()

2019-12-04 14:10:46 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:10:47 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:10:50 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:10:51 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:10:52 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 14:10:55 Hail: INFO: Ordering unsorted dataset with network shuffle


locus,alleles,prs_A1,adj_beta,adj_beta_flipped
locus<GRCh37>,array<str>,str,float64,float64
22:18080154,"[""A"",""G""]","""A""",9.24e-02,9.24e-02
22:18319179,"[""C"",""T""]","""C""",1.67e-02,1.67e-02
22:19030113,"[""A"",""G""]","""A""",2.03e-02,2.03e-02
22:19197949,"[""G"",""A""]","""G""",-6.10e-02,-6.10e-02
22:19232466,"[""G"",""A""]","""G""",9.60e-02,9.60e-02
22:19506456,"[""G"",""T""]","""G""",1.95e-02,1.95e-02
22:19665133,"[""G"",""A""]","""G""",-5.93e-02,-5.93e-02
22:19690250,"[""A"",""G""]","""A""",-7.59e-02,-7.59e-02
22:19921378,"[""C"",""T""]","""C""",5.89e-02,5.89e-02


In [19]:
r_alt = sim_cols.aggregate(hl.agg.corr(sim_cols.y, sim_cols.y_no_noise))

In [20]:
r_alt

0.8366737392792422

In [21]:
r_alt**2

0.7000229459995093

In [16]:
sim_cols.select('y','y_no_noise').show()

s,y,y_no_noise
str,float64,float64
"""1000019""",3.97e-01,2.99e-01
"""1000022""",-1.12e+00,-1.01e+00
"""1000035""",-1.01e+00,-5.21e-01
"""1000046""",1.47e+00,1.03e+00
"""1000054""",-9.07e-01,-3.13e-02
"""1000063""",1.65e+00,9.57e-01
"""1000081""",9.53e-01,4.89e-01
"""1000090""",-1.32e+00,-1.17e+00
"""1000105""",3.51e-01,5.73e-02


In [26]:
test1.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'isFemale': bool
    'age': int32
    'age_squared': int32
    'age_isFemale': int32
    'age_squared_isFemale': int32
    'PC1': float64
    'PC2': float64
    'PC3': float64
    'PC4': float64
    'PC5': float64
    'PC6': float64
    'PC7': float64
    'PC8': float64
    'PC9': float64
    'PC10': float64
    'PC11': float64
    'PC12': float64
    'PC13': float64
    'PC14': float64
    'PC15': float64
    'PC16': float64
    'PC17': float64
    'PC18': float64
    'PC19': float64
    'PC20': float64
    'sim_y': float64
    'y_no_noise': float64
    'prs': float64
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'varid': str
    'A1_1kg': str
    'A2_1kg': str
    'AF': float64
    'adj_beta': float64
    'prs_A1': str
    'adj_beta_flipped': float64
-------------------

In [27]:
test1.cols().select('y_no_noise','prs').show()

2019-12-04 13:36:51 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 13:36:53 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 13:36:59 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 13:37:01 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 13:37:02 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-04 13:37:06 Hail: INFO: Ordering unsorted dataset with network shuffle


s,y_no_noise,prs
str,float64,float64
"""1000022""",7.76e-01,7.76e-01
"""1000046""",-9.36e-01,-9.36e-01
"""1000141""",1.14e-01,1.14e-01
"""1000198""",-1.04e+00,-1.04e+00
"""1000236""",-3.51e-01,-3.51e-01
"""1000255""",-1.74e+00,-1.74e+00
"""1000291""",4.77e-01,4.77e-01
"""1000396""",-1.22e+00,-1.22e+00
"""1000408""",-1.54e+00,-1.54e+00


In [ ]:
get_corr_pt(variant_set=variant_set, maf=maf, h2=h2, pi=pi,
            use_1kg_eur_hm3_snps=use_1kg_eur_hm3_snps,is_pruned=is_pruned,
            threshold=1,chrom=chrom,max_reps=60)

2019-12-03 22:19:18 Hail: INFO: Reading table with no type imputation
  Loading column 's' as type 'str' (type not specified)
  Loading column 'label_100000' as type 'str' (type not specified)
  Loading column 'label_50000' as type 'str' (type not specified)
  Loading column 'label_20000' as type 'str' (type not specified)
  Loading column 'label_10000' as type 'str' (type not specified)
  Loading column 'label_5000' as type 'str' (type not specified)




###############
test mt col count: 61144
###############

... Not pruning because is_pruned=False ...
----------------------------------------
Global fields:
    'ldscsim': struct {
        h2: float64, 
        pi: array<float64>
    } 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'SNP': str 
    'varid': str 
    'A1': str 
    'A2_1kg': str 
    'AF': float64 
    'true_beta': float64 
----------------------------------------
Key: ['SNP']
----------------------------------------
None


2019-12-03 22:19:30 Hail: INFO: Reading table to impute column types
2019-12-03 22:19:30 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'BETA' as type 'float64' (imputed)
  Loading column 'P' as type 'float64' (imputed)
2019-12-03 22:19:52 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-03 22:19:52 Hail: INFO: Coerced sorted dataset
2019-12-03 22:19:56 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-03 22:20:08 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-03 22:20:10 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-03 22:20:10 Hail: INFO: Coerced sorted dataset


In [8]:
tb.beta.show()

locus,alleles,beta
locus<GRCh37>,array<str>,float64
22:16886873,"[""G"",""A""]",-0.00e+00
22:16892858,"[""A"",""G""]",0.00e+00
22:17054720,"[""T"",""C""]",0.00e+00
22:17056415,"[""T"",""C""]",0.00e+00
22:17057138,"[""G"",""T""]",-0.00e+00
22:17058616,"[""C"",""T""]",0.00e+00
22:17067504,"[""G"",""T""]",-0.00e+00
22:17072483,"[""A"",""G""]",-0.00e+00
22:17074622,"[""A"",""C""]",0.00e+00
